In [1]:
import opvious.modeling as om

class Sudoku(om.Model):
    """A mixed-integer model for Sudoku"""
    
    positions = om.interval(0, 8, name='P')
    values = om.interval(1, 9, name='V')

    def __init__(self):
        self.input = om.Parameter.indicator(self.grid * self.values, qualifiers=['row', 'column', 'value'])
        self.output = om.Variable.indicator(self.grid * self.values, qualifiers=['row', 'column', 'value'])

    @property
    def grid(self):
        """Cross-product of (row, column) positions"""
        return self.positions * self.positions

    @om.constraint
    def output_matches_input(self):
        """The output must match all input values where specified"""
        for i, j, v in self.grid * self.values:
            if self.input(i, j, v):
                yield self.output(i, j, v) >= self.input(i, j, v)

    @om.constraint
    def one_output_per_cell(self):
        """Each cell has exactly one value"""
        for i, j in self.grid:
            yield om.total(self.output(i, j, v) == 1 for v in self.values)

    @om.constraint
    def one_value_per_column(self):
        """Each value is present exactly once per column"""
        for j, v in self.positions * self.values:
            yield om.total(self.output(i, j, v) == 1 for i in self.positions)

    @om.constraint
    def one_value_per_row(self):
        """Each value is present exactly once per row"""
        for i, v in self.positions * self.values:
            yield om.total(self.output(i, j, v) == 1 for j in self.positions)

    @om.constraint
    def one_value_per_box(self):
        """Each value is present exactly once per box"""
        for v, b in self.values * self.positions:
            yield om.total(
                self.output(3 * (b // 3) + c // 3, 3 * (b % 3) + c % 3, v) == 1
                for c in self.positions
            )
            
model = Sudoku()
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Sudoku</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^p_\mathrm{input[row,column,value]}&: i \in \{0, 1\}^{P \times P \times V} \\
  \S^v_\mathrm{output[row,column,value]}&: \omicron \in \{0, 1\}^{P \times P \times V} \\
  \S^a&: P \doteq \{ 0 \ldots 8 \} \\
  \S^a&: V \doteq \{ 1 \ldots 9 \} \\
  \S^c_\mathrm{outputMatchesInput}&: \forall p \in P, p' \in P, v \in V \mid i_{p,p',v} \neq 0, \omicron_{p,p',v} \geq i_{p,p',v} \\
  \S^c_\mathrm{oneOutputPerCell}&: \forall p \in P, p' \in P, \sum_{v \in V} \omicron_{p,p',v} = 1 \\
  \S^c_\mathrm{oneValuePerColumn}&: \forall p \in P, v \in V, \sum_{p' \in P} \omicron_{p',p,v} = 1 \\
  \S^c_\mathrm{oneValuePerRow}&: \forall p \in P, v \in V, \sum_{p' \in P} \omicron_{p,p',v} = 1 \\
  \S^c_\mathrm{oneValuePerBox}&: \forall v \in V, p \in P, \sum_{p' \in P} \omicron_{3 \left\lfloor \frac{p}{3} \right\rfloor + \left\lfloor \frac{p'}{3} \right\rfloor,3 \left(p \bmod 3\right) + p' \bmod 3,v} = 1 \\
\end{align*}
$$
</div>
</details>
</div>